In [1]:
import os
import pyrtl
from pyrtl import WireVector, Input, Output, CompiledSimulation, reset_working_block
import torch
from typing import Literal
from itertools import product
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss
from hardware_accelerators.dtypes import *
from hardware_accelerators.simulation import get_sim_cache_dir
from hardware_accelerators.simulation.accelerator import CompiledAcceleratorSimulator
from hardware_accelerators.rtllib.accelerator import CompiledAcceleratorConfig
from hardware_accelerators.rtllib.multipliers import *
from hardware_accelerators.rtllib.lmul import *
from hardware_accelerators.nn import load_model, MLP

model = load_model("models/mlp_mnist.pth")

In [ ]:
docker run -d --name sc_server \
  -p 8085:8085 \
  --restart unless-stopped \
  ghcr.io/siliconcompiler/sc_runner:latest \
  sc-server -port 8085 -loglevel info -auth false -nfsmount /workdir

In [ ]:
from siliconcompiler import Chip
from siliconcompiler.targets import asap7_demo


chip = Chip("pe_pipeline_fast_lmul")  # create chip object
chip.input("verilog/pe/w16a16/pe_fast_pipelined_lmul.v")
chip.clock("clk", period=1)  # define clock speed of design
chip.use(asap7_demo)  # load predefined technology and flow target
chip.set("option", "remote", True)  # run remote in the cloud
chip.run()  # run compilation of design and target
chip.summary()  # print results summary

| INFO    | verilog/pe/w16a16/pe_fast_pipelined_lmul.v inferred as rtl/verilog
| INFO    | job0 | -----                | - | Using credentials: /Users/kaibreese/.sc/credentials
| INFO    | job0 | -----                | - | Collecting input sources
| INFO    | job0 | -----                | - | Copying /Users/kaibreese/UCSD/dsc180b/hardware-accelerators/verilog/pe/w16a16/pe_fast_pipelined_lmul.v to '/Users/kaibreese/UCSD/dsc180b/hardware-accelerators/build/pe_pipeline_fast_lmul/job0/sc_collected_files' directory
| INFO    | job0 | remote               | - | Starting job: 7befdbebf8d64af38d97bdc64c91277c
| INFO    | job0 | remote               | - | Your job's reference ID is: 7befdbebf8d64af38d97bdc64c91277c
| WARNING | job0 | remote               | 0 | Job is still running: Expecting value: line 1 column 1 (char 0)
| WARNING | job0 | remote               | 0 | Job is still running: Expecting value: line 1 column 1 (char 0)
| WARNING | job0 | remote               | 0 | Job is still runni

Empty file returned from remote for: import.chisel0
Empty file returned from remote for: import.combine0
Empty file returned from remote for: import.vhdl0
Empty file returned from remote for: import.c0
Empty file returned from remote for: import.convert0
Empty file returned from remote for: import.bluespec0
Empty file returned from remote for: import.bluespec0


----------------------------------------------------------------------------
SUMMARY:

design : pe_pipeline_fast_lmul
params : None
jobdir : /Users/kaibreese/UCSD/dsc180b/hardware-accelerators/build/pe_pipeline_fast_lmul/job0
foundry : virtual
process : asap7
targetlibs : asap7sc7p5t_rvt 

                      import.verilog0         syn0        floorplan.init0  \
errors                       0                 0                 0          
warnings                     1                 2                 36         
drvs                        ---               ---                0          
drcs                        ---               ---               ---         
unconstrained               ---               ---                66         
cellarea       um^2         ---             174.275           170.119       
totalarea      um^2         ---               ---             523.174       
macroarea      um^2         ---               ---              0.000        
padcellarea    u

## Generating configs


In [15]:
def generate_test_configs():
    configs = {}
    NN_TEST_MUL_FNS = [float_multiplier, lmul_fix]
    NN_TEST_WA_DTYPES = [
        (Float8, Float8),
        (Float8, BF16),
        (Float8, Float32),
        (BF16, BF16),
        (BF16, Float32),
        (Float32, Float32),
    ]
    for dtypes, mult_fn in product(NN_TEST_WA_DTYPES, NN_TEST_MUL_FNS):
        weight_type, act_type = dtypes
        mul_name = "lmul" if "lmul" in mult_fn.__name__ else ""
        config = CompiledAcceleratorConfig(
            array_size=8,
            weight_type=weight_type,
            activation_type=act_type,
            multiplier=mult_fn,
        )
        configs[f"w{weight_type.bitwidth()}a{act_type.bitwidth()}{mul_name}"] = config

    return configs


configs = generate_test_configs()

list(configs.keys())

['w8a8',
 'w8a8lmul',
 'w8a16',
 'w8a16lmul',
 'w8a32',
 'w8a32lmul',
 'w16a16',
 'w16a16lmul',
 'w16a32',
 'w16a32lmul',
 'w32a32',
 'w32a32lmul']

## Test loading from saved sim


In [5]:
# Data transformation: convert images to tensor and normalize them
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
    ]
)
# Download MNIST test data
test_dataset = datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


def get_batch(batch_size):
    loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    batch, labels = next(iter(loader))
    return batch.reshape(batch_size, -1).numpy(), labels.numpy()


def get_activation():
    image, _ = next(iter(test_loader))
    image = image.detach().numpy().reshape(-1)
    return image

# Testing compiled sims


In [4]:
lmul_config = CompiledAcceleratorConfig(
    array_size=8,
    weight_type=BF16,
    activation_type=BF16,
    multiplier=lmul_fix,
)
reset_working_block()
lmul_sim = CompiledAcceleratorSimulator(lmul_config, model, recompile=True)

In [18]:
batch, labels = get_batch(5)

results = lmul_sim.predict_batch(batch, apply_softmax=False)

print(np.array2string(results["hidden_out"], precision=3, suppress_small=True))

print(labels)

[[ 0.     0.     3.109  0.     0.     7.     6.469  0.     0.     2.562
   3.578  0.     0.     0.     0.     0.     0.     0.     0.     0.
   0.     0.     4.031  0.     0.     0.     4.25   7.094  0.     0.
   0.     2.594  0.     4.281  0.357  1.141  0.     1.266  0.     0.
   5.094  0.     0.     0.     0.     9.25   0.     0.     0.     0.
   0.     8.562  0.     6.75   0.     0.     0.     0.     0.     0.
   0.699  0.     0.     0.     0.     0.404  0.     0.     0.     0.
   0.     8.312  2.234  0.801  0.     0.     0.     0.     0.     0.816
   0.     0.     2.953  2.547  9.625  8.125  0.     0.     0.     0.
   0.     4.656 10.75   0.     0.57   4.125  0.     2.453  0.    10.812
   0.     0.     4.094  0.     0.984  6.844  0.     1.562  0.     0.707
   0.     7.188  2.109  0.     4.344  0.     0.    11.75   1.078  0.
   0.     0.     0.     0.     2.969  0.     0.     8.125]
 [ 0.    10.     0.     0.     1.883  0.     0.     5.25   2.578  0.
   0.     3.906  0.    13.25   0

In [ ]:
lmul_sim.

In [19]:
results["final_out"]

array([[-9.30459597e+36, -1.06338240e+37, -5.31691198e+36,
        -9.30459597e+36, -5.31691198e+36, -3.98768399e+36,
        -6.64613998e+36, -7.97536797e+36, -7.97536797e+36,
        -7.97536797e+36],
       [-7.97536797e+36, -1.06338240e+37, -1.19630520e+37,
        -5.31691198e+36, -6.64613998e+36, -7.97536797e+36,
        -7.97536797e+36, -5.31691198e+36, -9.30459597e+36,
        -5.31691198e+36],
       [-6.64613998e+36, -1.46215080e+37, -1.06338240e+37,
        -7.97536797e+36, -7.97536797e+36, -6.64613998e+36,
        -6.64613998e+36, -9.30459597e+36, -7.97536797e+36,
        -5.31691198e+36],
       [-1.46215080e+37, -1.19630520e+37, -5.31691198e+36,
        -7.97536797e+36, -9.30459597e+36, -6.64613998e+36,
        -1.06338240e+37, -6.64613998e+36, -1.06338240e+37,
        -5.31691198e+36],
       [-1.06338240e+37, -9.30459597e+36, -3.98768399e+36,
        -7.97536797e+36, -1.06338240e+37, -3.98768399e+36,
        -1.06338240e+37, -3.98768399e+36, -1.19630520e+37,
        -7.

In [14]:
act = get_activation()

logits = lmul_sim.predict(act)

print(np.array2string(logits, precision=3, suppress_small=True))
print(np.argmax(logits))

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
5


In [6]:
reset_working_block()

In [ ]:
sim_w8a16 = CompiledAcceleratorSimulator(configs["w8a16"], model=model)
sim_w16a16 = CompiledAcceleratorSimulator(configs["w16a16"], model=model)

In [16]:
sim_w8a16lmul = CompiledAcceleratorSimulator(
    configs["w8a16lmul"], model=model, recompile=True
)

In [20]:
pyrtl.set_debug_mode(False)

# Running batch inference


In [17]:
batch, labels = get_batch(5)

results = sim_w8a16lmul.predict_batch(batch, apply_softmax=False)

# logits_str = np.array2string(logits, precision=4, suppress_small=True)
# print(logits_str)

results["hidden_out"], results["final_out"]

(array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00,            nan,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
                    nan, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00,            nan,
         0.00000000e+00, 0.00000000e+00,            nan, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00,            nan,
         0.00000000e+00, 0.00000000e+00, 5.31691198e+37, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000

In [63]:
print(np.array2string(results["final_out"], precision=4, suppress_small=True))

[[-9.3046e+36 -1.0634e+37 -5.3169e+36 -9.3046e+36 -5.3169e+36 -3.9877e+36
  -6.6461e+36 -7.9754e+36 -7.9754e+36 -7.9754e+36]
 [-7.9754e+36 -1.0634e+37 -1.1963e+37 -5.3169e+36 -6.6461e+36 -7.9754e+36
  -7.9754e+36 -5.3169e+36 -9.3046e+36 -5.3169e+36]
 [-6.6461e+36 -1.4622e+37 -1.0634e+37 -7.9754e+36 -7.9754e+36 -6.6461e+36
  -6.6461e+36 -9.3046e+36 -7.9754e+36 -5.3169e+36]
 [-1.4622e+37 -1.1963e+37 -5.3169e+36 -7.9754e+36 -9.3046e+36 -6.6461e+36
  -1.0634e+37 -6.6461e+36 -1.0634e+37 -5.3169e+36]
 [-1.0634e+37 -9.3046e+36 -3.9877e+36 -7.9754e+36 -1.0634e+37 -3.9877e+36
  -1.0634e+37 -3.9877e+36 -1.1963e+37 -7.9754e+36]]


In [36]:
batch, labels = get_batch(1000)

results = sim_w16a16.predict_batch(batch)
preds = np.argmax(results, axis=1)

print()
print(preds)
print(labels)
print(preds == labels)

KeyboardInterrupt: 

In [25]:
config_w16a16s4 = CompiledAcceleratorConfig(
    array_size=4,
    activation_type=BF16,
    weight_type=BF16,
    multiplier=float_multiplier,
)

sim_w16a16s4 = CompiledAcceleratorSimulator(config_w16a16s4, model=model)

Loading existing binary for config wb16ab16-4x4 from /Users/kaibreese/.hardware_accelerators/sim_cache/b58071ffdd2c4c88


In [26]:
batch, labels = get_batch(10)

results = sim_w16a16s4.predict_batch(batch)
preds = np.argmax(results, axis=1)

print()
print(preds)
print(labels)
print(preds == labels)

Completed 6304/6403 tiles
[7 2 1 0 4 1 4 9 5 9]
[7 2 1 0 4 1 4 9 5 9]
[ True  True  True  True  True  True  True  True  True  True]


In [33]:
pred = sim_w8a16.predict(get_activation())
np.argmax(pred).item(), len(pred)

(7, 10)

In [32]:
pred = sim_w16a16.predict(get_activation())
np.argmax(pred).item(), len(pred)

(7, 10)

In [34]:
batch, labels = get_batch(10)

results = sim_w8a16.predict_batch(batch)
preds = np.argmax(results, axis=1)

print()
print(preds)
print(labels)
print(preds == labels)

Completed 1584/1618 tiles
[7 2 1 0 4 1 4 9 5 9]
[7 2 1 0 4 1 4 9 5 9]
[ True  True  True  True  True  True  True  True  True  True]


In [28]:
batch, labels = get_batch(10)

results = sim_fp8.predict_batch(batch)
preds = np.argmax(results, axis=1)

print()
print(preds)
print(labels)
print(preds == labels)

Completed 1584/1618 tiles
[7 0 7 0 3 0 2 0 9 0]
[7 2 1 0 4 1 4 9 5 9]
[ True False False  True False False False False False False]


In [ ]:
print(sim_bf16.config)

CompiledAcceleratorConfig(
        array_size: 8
        activation_type: BF16
        weight_type: BF16
        multiplier: IEEE 754
        accum_addr_width: 12
        pipeline: False
        name: wb16ab16s8
    )


In [79]:
def get_batch(batch_size):
    loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    batch, labels = next(iter(loader))
    return batch.reshape(batch_size, -1).numpy(), labels


batch, labels = get_batch(10)
preds == labels

tensor([True, True, True, True, True, True, True, True, True, True])

In [106]:
isinstance(model, torch.nn.Module)

True

In [84]:
torch.tensor(results, dtype=torch.bfloat16)

tensor([[7.4215e-09, 7.4465e-12, 4.8801e-07, 6.7055e-06, 3.6637e-14, 3.8199e-10,
         9.2704e-15, 1.0000e+00, 1.1350e-09, 4.7730e-08],
        [1.3853e-08, 3.4332e-05, 1.0000e+00, 1.1176e-06, 4.6629e-15, 1.6917e-10,
         3.8417e-09, 4.4587e-13, 9.1735e-08, 2.2471e-13],
        [1.8394e-08, 1.0000e+00, 1.3351e-03, 6.3896e-05, 6.5804e-05, 4.1246e-05,
         2.9951e-06, 2.9206e-05, 7.4863e-05, 8.3074e-07],
        [1.0000e+00, 1.1539e-11, 3.4051e-09, 7.5033e-11, 2.0140e-08, 3.4226e-08,
         2.2352e-07, 2.0373e-10, 4.6020e-10, 3.5912e-06],
        [1.5061e-09, 7.0486e-11, 6.4261e-08, 5.4797e-11, 1.0000e+00, 6.1933e-08,
         1.7812e-08, 5.1036e-07, 4.2201e-09, 5.3406e-04],
        [1.6080e-09, 1.0000e+00, 1.0967e-05, 3.5018e-06, 1.8954e-05, 7.1886e-09,
         4.3656e-09, 3.2425e-05, 1.0058e-06, 4.1444e-08],
        [7.7716e-14, 2.7753e-07, 3.8445e-06, 1.5497e-06, 1.0000e+00, 1.3828e-05,
         2.0838e-08, 5.8711e-06, 8.8120e-04, 1.2207e-04],
        [9.5952e-11, 3.7812

In [92]:
np.sum(results, axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [ ]:
CrossEntropyLoss()(torch.tensor(results), labels)

1.4622700768294972

In [ ]:
# Evaluation function
def evaluate(
    sim: CompiledAcceleratorSimulator,
    dataset,
    batch_size,
    criterion=CrossEntropyLoss(),
):
    if not sim.model_loaded:
        raise Warning(f"No model loaded for sim {sim.config.name}.")
    correct = 0
    total = 0
    running_loss = 0.0
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    for batch, labels in data_loader:
        batch = batch.reshape(batch_size, -1).numpy()
        outputs = sim.predict_batch(batch)
        loss = criterion(torch.tensor(outputs), labels)
        running_loss += loss.item()

        # Get predictions from the maximum value
        predicted = np.argmax(outputs, axis=1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        break
    avg_loss = running_loss / len(data_loader)
    accuracy = 100.0 * correct / total
    return avg_loss, accuracy


evaluate(sim_bf16, model, test_dataset, 10)

[array([[0.      , 0.      , 3.109375, 0.      ],
       [0.      , 9.9375  , 0.      , 0.      ],
       [0.      , 0.      , 0.      , 0.      ],
       [0.      , 8.0625  , 3.6875  , 3.078125],
       [0.      , 7.625   , 0.546875, 0.      ],
       [0.      , 0.      , 0.      , 0.      ],
       [0.      , 2.609375, 1.15625 , 0.      ],
       [0.      , 0.      , 0.      , 0.      ],
       [0.      , 9.5625  , 0.      , 0.      ],
       [0.      , 7.375   , 0.      , 3.78125 ]]), array([[ 0.      ,  6.8125  ,  6.625   ,  0.      ],
       [ 1.9375  ,  0.      ,  0.      ,  4.9375  ],
       [ 0.      ,  0.      ,  0.      ,  6.8125  ],
       [ 0.      ,  0.      ,  6.9375  ,  0.      ],
       [ 0.      ,  5.3125  , 11.8125  ,  0.      ],
       [ 0.      ,  1.6875  ,  0.      ,  6.0625  ],
       [ 0.      ,  1.078125,  8.5625  ,  0.      ],
       [ 0.      ,  3.96875 ,  3.96875 ,  4.0625  ],
       [ 0.      ,  0.      , 10.6875  ,  0.      ],
       [ 0.      ,  7.6875  , 

(0.001462270076829497, 100.0)

In [ ]:
from hardware_accelerators.simulation.matrix_utils import bias_trick


weights_1 = model.fc1.weight.numpy(force=True)
bias_1 = model.fc1.bias.numpy(force=True)
weights_2 = model.fc2.weight.numpy(force=True)
bias_2 = model.fc2.bias.numpy(force=True)

# Apply the bias trick
W1_aug = bias_trick(weights_1, bias_1)
W2_aug = bias_trick(weights_2, bias_2)

W1_aug.shape[0]

128

In [ ]:
10000 / 10

1000.0

# Compiling all configurations


In [ ]:
from typing import Iterator, Type, List, Callable
from itertools import product

from hardware_accelerators.dtypes import *


def generate_accelerator_configs(
    array_size: int = 16,
    dtypes: List[Type[BaseFloat]] = None,
    multipliers: List[Callable] = None,
) -> Iterator[CompiledAcceleratorConfig]:
    """
    Generate all valid CompiledAcceleratorConfig combinations.

    Args:
        array_size: Size of the systolic array
        dtypes: List of data types to consider. Defaults to [Float8, BF16, FP16, FP32]
        multipliers: List of multiplier functions. Defaults to [float_multiplier, lmul]

    Yields:
        Valid CompiledAcceleratorConfig objects

    Restrictions:
        1. The activation_type must be greater than or equal to the weight_type in terms of bitwidth.
        2. 16-bit float types (BF16, FP16) should not be combined with each other.
           They should only pair with themselves or with FP32.
    """
    if dtypes is None:
        dtypes = [Float8, BF16, Float16, Float32]

    if multipliers is None:
        multipliers = [float_multiplier, lmul_fast]

    # Sort dtypes by bitwidth for easier comparison
    dtype_bitwidths = {dtype: dtype.bitwidth() for dtype in dtypes}
    sorted_dtypes = sorted(dtypes, key=lambda d: dtype_bitwidths[d])

    # Identify 16-bit float types
    bit16_float_types = [dtype for dtype in dtypes if dtype_bitwidths[dtype] == 16]

    # Generate all combinations
    for multiplier in multipliers:
        for weight_type in sorted_dtypes:
            # Find valid activation types based on bitwidth
            valid_activation_types = [
                dtype
                for dtype in sorted_dtypes
                if dtype_bitwidths[dtype] >= dtype_bitwidths[weight_type]
            ]

            for activation_type in valid_activation_types:
                # Skip invalid combinations of 16-bit float types
                if (
                    weight_type in bit16_float_types
                    and activation_type in bit16_float_types
                    and weight_type != activation_type
                ):
                    continue

                yield CompiledAcceleratorConfig(
                    array_size=array_size,
                    activation_type=activation_type,
                    weight_type=weight_type,
                    multiplier=multiplier,
                )


# Example usage:
def print_all_configs():
    for i, config in enumerate(generate_accelerator_configs()):
        print(f"Config {i+1}:")
        print(f"  Array Size: {config.array_size}")
        print(f"  Activation Type: {config.activation_type.__name__}")
        print(f"  Weight Type: {config.weight_type.__name__}")
        print(f"  Multiplier: {config.multiplier.__name__}")
        print()

In [32]:
print_all_configs()

Config 1:
  Array Size: 16
  Activation Type: Float8
  Weight Type: Float8
  Multiplier: float_multiplier

Config 2:
  Array Size: 16
  Activation Type: BF16
  Weight Type: Float8
  Multiplier: float_multiplier

Config 3:
  Array Size: 16
  Activation Type: Float16
  Weight Type: Float8
  Multiplier: float_multiplier

Config 4:
  Array Size: 16
  Activation Type: Float32
  Weight Type: Float8
  Multiplier: float_multiplier

Config 5:
  Array Size: 16
  Activation Type: BF16
  Weight Type: BF16
  Multiplier: float_multiplier

Config 6:
  Array Size: 16
  Activation Type: Float32
  Weight Type: BF16
  Multiplier: float_multiplier

Config 7:
  Array Size: 16
  Activation Type: Float16
  Weight Type: Float16
  Multiplier: float_multiplier

Config 8:
  Array Size: 16
  Activation Type: Float32
  Weight Type: Float16
  Multiplier: float_multiplier

Config 9:
  Array Size: 16
  Activation Type: Float32
  Weight Type: Float32
  Multiplier: float_multiplier

Config 10:
  Array Size: 16
  Activa